In [1]:
#https://github.com/alexeyev/abae-pytorch 

# TODO pulisci questo file
# todo  prova con is_alpha in preprocessing e buttiamo via il resto (con sequenze piu lunghe)!
#  --> leggi top 1000 di restaruant e vedi split con '' quanto lunghe in media
# todo fix regularization sia su matrix che generation di aspects / emb

# https://arxiv.org/pdf/1803.09820
# https://machinelearningmastery.com/learning-rate-for-deep-learning-neural-networks/

In [2]:
# Una considerazione: Potrei sfruttare gli aspetti inferiti da quello che ho per filtrare via dal testo documenti che parlano di "Piattaforme", "Kickstarter", "Materiali" visto che queste le becca con facilita.
# Il mio dataset é probabilmente troppo vario al momento. Un filtraggio di questo tipo faciliterebbe l'apprendimento.

# Filtro a mano i commenti scaricati per togliere quelli poco significativi? Riduco rumore!

## Regularization
>We hope to learn vector representations of the most representative aspects for a review dataset. <br>
However, the aspect embedding matrix T may suffer from redundancy problems during training. [...] <br>
> The regularization term encourages orthogonality among the rows of the aspect embedding matrix T and penalizes redundancy between different aspect vectors <br>
> ~ Ruidan

We use an Orthogonal Regulizer definition of the method can be found here: https://paperswithcode.com/method/orthogonal-regularization. <br/>
For the code we use the default implementation provided by Keras (https://keras.io/api/layers/regularizers/)

## Hands on first attempt:


In [3]:
# Choice of max_seq size is relative to the ds. 95th percentile (at max 5% loss of information)

In [4]:
from core.dataset import PandasPositiveNegativeNumericTextDataset
from core.train import ABAEModelConfiguration, ABAEModelManager
import pandas as pd

corpus = "../output/dataset/default-pre-processed/80k.preprocessed.csv"
config = ABAEModelConfiguration(corpus_file=corpus, model_name=f"ff")
config.epochs = 7  # Just one epoch
config.max_sequence_length = 40
config.aspect_size = 16
config.negative_sample_size = 20
print(f"Running on default config:\n {config}")

# Without any hp tuning we just try and see how it goes.
manager = ABAEModelManager(config)
train_dataset = PandasPositiveNegativeNumericTextDataset(
    dataframe=pd.read_csv(corpus),
    vocabulary=manager.embedding_model.vocabulary(),
    negative_size=config.negative_sample_size,
    max_seq_length=config.max_sequence_length
)

Running on default config:
 ABAEModelConfiguration(corpus_file='../output/dataset/default-pre-processed/80k.preprocessed.csv', model_name='ff', aspect_size=16, max_vocab_size=None, embedding_size=200, learning_rate=0.01, decay_rate=0.95, momentum=0.9, max_sequence_length=40, negative_sample_size=20, batch_size=64, epochs=7, output_path='./output')


KeyboardInterrupt: 

In [2]:
manager.run_train_process(train_dataset, optimizer='adam')

1473/1473 ━━━━━━━━━━━━━━━━━━━━ 228s 154ms/step - loss: 8.6859 - max_margin_loss: 8.6790


(<keras.src.callbacks.history.History at 0x29e671cf350>,
 <Functional name=functional_1, built=True>)

In [4]:
iteration_model = manager.get_training_model()

In [5]:
import torch
# THIS IS OKAY!
from gensim import corpora
# See if another coherence metric might be better.
from core.evaluation import coherence_model_generation, get_aspect_top_k_words
from core.dataset import TokenizedDataset
from core.train import ABAEModelManager, ABAEModelConfiguration
from keras import ops as K

# word_emb = normalize(iteration_model.get_layer('word_embedding').weights[0].value.data)
# aspect_embeddings = normalize(iteration_model.get_layer('aspect_embedding').w)
word_emb = iteration_model.get_layer('word_embeddings').weights[0].value.data
aspect_embeddings = iteration_model.get_layer('aspect_embeddings').w

word_emb = torch.nn.functional.normalize(word_emb, p=2, dim=-1)
aspect_embeddings = torch.nn.functional.normalize(aspect_embeddings, p=2, dim=-1)

inv_vocab = manager.embedding_model.model.wv.index_to_key

aspects_top_k_words = [get_aspect_top_k_words(a, word_emb, inv_vocab, top_k=50) for a in aspect_embeddings]
aspect_words = [[word[0] for word in aspect] for aspect in aspects_top_k_words]  # Remap

ds = TokenizedDataset(corpus, manager.embedding_model.vocabulary())
dictionary = corpora.Dictionary(ds.text_ds.apply(lambda x: x.split(' ')).to_list())

_, m = coherence_model_generation(aspect_words, ds.text_ds.apply(lambda x: x.split(' ')), dictionary, topn=3)

m.get_coherence()

D:\PycharmProjects\nlp-course-project\core\evaluation.py:56: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3687.)
  similarity = word_embeddings @ aspect.T



Given aspect most representative words are:
Word:  rule (0.6814370155334473)
Word:  rulebook (0.6234116554260254)
Word:  manual (0.6020396947860718)
Word:  reference (0.5943478345870972)
Word:  instruction (0.5853652358055115)
Word:  faq (0.5512462854385376)
Word:  forum (0.5474805235862732)
Word:  page (0.5156159996986389)
Word:  guide (0.5124691128730774)
Word:  clarification (0.5111934542655945)
Word:  paragraph (0.48346424102783203)
Word:  answer (0.4652719795703888)
Word:  booklet (0.4625762701034546)
Word:  ambiguous (0.4545668065547943)
Word:  errata (0.45331618189811707)
Word:  aid (0.44917163252830505)
Word:  file (0.4448372721672058)
Word:  poorly (0.443092405796051)
Word:  translation (0.4428563714027405)
Word:  refer (0.44281649589538574)
Word:  download (0.4397127330303192)
Word:  helpful (0.43926429748535156)
Word:  sheet (0.4332764744758606)
Word:  book (0.4292224049568176)
Word:  iconography (0.42272233963012695)
Word:  confusing (0.4203360974788666)
Word:  tutorial (0

Pandas Apply:   0%|          | 0/94248 [00:00<?, ?it/s]

Max sequence length calculation in progress...
Max sequence length is:  416 . The limit is set to 80 tokens.
We loose information on 21 points.This is 0.022281639928698752% of the dataset.
Padding sequences to length (80).


INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary<7882 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #20000 to Dictionary<11292 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #30000 to Dictionary<13709 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #40000 to Dictionary<15621 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #50000 to Dictionary<17363 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #60000 to Dictionary<18957 unique tokens: ['game', 'german', 'involve', 'long', 'politic']...>
INFO:gensim.corpora.dictionary:adding document #70000 to Diction

-0.06975844388455707

In [6]:
m.get_coherence_per_topic()

[-0.07990060733246114,
 0.2743863220801869,
 -0.039693072474737336,
 -0.06331046435950176,
 -0.2756276683915752,
 -0.024395088608448762,
 -0.0701294019252142,
 -0.1982382135709934,
 -0.03980300302191894,
 0.4425553892722734,
 -0.12215550305377003,
 -0.2685362188016142,
 -0.48079022183623965,
 0.08988606782609228,
 -0.5107417881005712,
 0.2503583701455803]

In [ ]:
from core.evaluation import coherence_per_aspect

aspect_words = [[word[0] for word in aspect] for aspect in aspects_top_k_words]
c, m = coherence_per_aspect(aspect_words, ds.text_ds, 3)

In [ ]:
c

In [ ]:
m.get_coherence()

In [ ]:
# THIS IS OKAY!
from gensim import corpora
# See if another coherence metric might be better.
from core.evaluation import coherence_model_generation, get_aspect_top_k_words
from core.dataset import TokenizedDataset
from core.train import ABAEModelManager, ABAEModelConfiguration

c_file = "../output/dataset/pre-processed/tuning.preprocessed.csv"
# Load aspects
config = ABAEModelConfiguration(corpus_file=corpus, model_name=f"hands_on")

manager = ABAEModelManager(config)
iteration_model = manager.get_inference_model()

# word_emb = normalize(iteration_model.get_layer('word_embedding').weights[0].value.data)
# aspect_embeddings = normalize(iteration_model.get_layer('aspect_embedding').w)
word_emb = iteration_model.get_layer('word_embedding').weights[0].value.data
aspect_embeddings = iteration_model.get_layer('aspect_embedding').w

inv_vocab = manager.embedding_model.model.wv.index_to_key

aspects_top_k_words = [get_aspect_top_k_words(a, word_emb, inv_vocab, top_k=3) for a in aspect_embeddings]
aspect_words = [[word[0] for word in aspect] for aspect in aspects_top_k_words]  # Remap

ds = TokenizedDataset(c_file, manager.embedding_model.vocabulary())
dictionary = corpora.Dictionary(ds.text_ds.apply(lambda x: x.split(' ')).to_list())

_, m = coherence_model_generation(aspect_words, ds.text_ds.apply(lambda x: x.split(' ')), dictionary, topn=3)

m.get_coherence()

## Aspect Embedding Size
The aspect embedding size is what will be inferring aspects. It is closest to representative words (?). <br />
We have to identify 7 actual aspects (luck, bookkeeping, downtime...) but that does not mean our matrix should be limited to rows only! <br>

For the first try we setup the aspect_size:
>The optimal number of rows is problem-dependent, so it’s crucial to: <br/>
> Start with a heuristic: Begin with 2–3x the number of aspects.

For **aspect extraction**, which involves identifying key aspects or topics in text, the best early stopping method depends on your approach:

### 1. Embedding-based Methods (e.g., Clustering Embeddings)
- **Silhouette Score**: Measure the separation and compactness of clusters. Stop when the score stabilizes.
- **Inertia/Distortion**: Track the sum of squared distances within clusters and stop when improvement flattens.
- **Centroid Movement**: Stop when the change in cluster centroids across iterations is minimal.

### 2. Topic Modeling (e.g., LDA)
- **Perplexity**: Monitor the perplexity on a held-out dataset and stop when it stops decreasing significantly.
- **Coherence Score**: Measure the semantic consistency of extracted topics and stop when it stabilizes.

### 3. Autoencoder-based Aspect Extraction
- **Reconstruction Loss**: Stop training when the validation reconstruction error no longer improves.

### 4. Qualitative Evaluation (if feasible)
- Periodically inspect extracted aspects for meaningfulness and diversity to decide on stopping.

For **aspect extraction**, combining an automated metric (like coherence score or silhouette score) with manual inspection often yields the best results.


## Hyperparameters Tuning
To tune our parameters we use a filtered version of the 50k ds. <br>
We filter out rows that can be found on the 200k ds.

In [ ]:
import pandas as pd

# This is based on the idea that our dataset are generated with different seeds else it won't work
large = pd.read_csv("../output/dataset/pre-processed/200k.preprocessed.csv")
small = pd.read_csv("../output/dataset/pre-processed/100k.preprocessed.csv")
tuning_set = small[~small["comments"].isin(large["comments"])]

tuning_set.to_csv("../output/dataset/pre-processed/tuning.preprocessed.csv", index=False)

> The main goal of ABAE is to extract interpretable and meaningful aspects, which makes coherence the more aligned metric.<br> Reconstruction error might help guide training but doesn’t guarantee that the extracted aspects are semantically useful.

In [ ]:
from core.hp_tuning import ABAERandomHyperparametersSelectionWrapper, HyperparameterTuningManager

configurations = 4  # We try 15 different configurations
corpus_file = "../output/dataset/pre-processed/tuning.preprocessed.csv"

print(f"Starting procedure. We try a total of {configurations}")
hp_wrapper = ABAERandomHyperparametersSelectionWrapper.create()
hp_tuning_manager = HyperparameterTuningManager(hp_wrapper, corpus_file, "./output")

In [ ]:
hp_tuning_manager(different_configurations=configurations, repeat=3) # todo tune not on metric but on loss. We want the best reconstruciton model.
# Bottom line: Prioritize classification performance, but monitor coherence if it affects usability.
# The output of this cell is very long therefore I deleted it

In [ ]:
from pathlib import Path
import json
import pandas as pd
import numpy as np

dataframe = pd.DataFrame(
    columns=[
        'name', 'mean_loss', 'mean_coherence', 'aspect_size', 'embedding_size', 'epochs', 'batch_size', 'learning_rate',
        'decay_rate', 'momentum', 'negative_sample_size'
    ]
)

i = 0
# Inspect results we stored for process. todo: Quando finisci salvalo come file da "tenere" in repo cosi che chi controlla lo ha gia.
for element in Path("./output").iterdir():

    # Search only for the tuning process runs.
    if element.is_dir() and element.name.startswith("tuning"):
        run_result = json.load(open(str(element) + "/run_results.json"))
        # We want the best configurations.

        obj = {
            'name': element.name,
            'mean_loss': np.mean(run_result["evaluation_loss"]),
            'mean_coherence': np.mean(run_result["coherence"])
        }
        temp = pd.DataFrame(run_result['params'] | obj, index=[i])
        dataframe = pd.concat([temp, dataframe])
        i += 1

In [ ]:
ds.text_ds.apply(lambda x: x.split(' '))

In [ ]:
# THIS IS OKAY!

from gensim import corpora
# See if another coherence metric might be better.
from core.evaluation import coherence_model_generation, normalize, get_aspect_top_k_words
import json
from core.dataset import TokenizedDataset
from core.train import ABAEModelManager, ABAEModelConfiguration

model_name = 'tuning_7128da5c-04c0-4e91-abf2-462f9646b48d'
c_file = "../output/dataset/pre-processed/tuning.preprocessed.csv"
# Load aspects
parameters = json.load(open(f"./output/{model_name}/run_results.json"))['params']
config = ABAEModelConfiguration(corpus_file=c_file, model_name=model_name, **parameters)

manager = ABAEModelManager(config)
iteration_model = manager.get_inference_model()

word_emb = normalize(iteration_model.get_layer('word_embedding').weights[0].value.data)
aspect_embeddings = normalize(iteration_model.get_layer('aspect_embedding').w)

# word_emb = iteration_model.get_layer('word_embedding').weights[0].value.data
# aspect_embeddings = iteration_model.get_layer('aspect_embedding').w

inv_vocab = manager.embedding_model.model.wv.index_to_key

aspects_top_k_words = [get_aspect_top_k_words(a, word_emb, inv_vocab, top_k=20) for a in aspect_embeddings]
aspect_words = [[word[0] for word in aspect] for aspect in aspects_top_k_words]  # Remap

ds = TokenizedDataset(c_file, manager.embedding_model.vocabulary())
dictionary = corpora.Dictionary(ds.text_ds.apply(lambda x: x.split(' ')).to_list())

_, m = coherence_model_generation(aspect_words, ds.text_ds.apply(lambda x: x.split(' ')), dictionary, topn=10)

m.get_coherence()

In [ ]:
manager.embedding_model.model.wv

In [ ]:
len(ds)

In [7]:
from torch.utils.data import DataLoader
from core.dataset import SimpleWord2VecEmbeddingsDataset

iteration_model = manager.get_inference_model()
custom_review = [
    'game hard fun', # 0
    'brother forever decide',
    'zero luck all skill',
    'lot downtime', # 3
    'i have banana',
    'plastic inserts',
    'complex game rules' # 6
]
# todo rebuild a prediciton model senza negative predict. Devo ristrutturare tutto1
ds = SimpleWord2VecEmbeddingsDataset(custom_review, manager.embedding_model.model)

res = iteration_model.predict(DataLoader(ds, batch_size=32))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [8]:
res[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.19045994, 0.20182857, 0.60771143],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0. 

In [9]:
# Il modello fa schifo non serve a niente.
res[1]  # Riddle correctly yields a classification to the aspect for representative.

array([[0.06208516, 0.03988646, 0.06572662, 0.07758018, 0.05268136,
        0.07338277, 0.07092224, 0.05171687, 0.05588178, 0.07390964,
        0.05801188, 0.05768691, 0.05894613, 0.06593319, 0.05774988,
        0.07789893],
       [0.04758715, 0.07573349, 0.04342123, 0.06475626, 0.05591242,
        0.05736869, 0.09891937, 0.08390705, 0.08602379, 0.06010805,
        0.06240946, 0.04144665, 0.06161851, 0.06739597, 0.04670087,
        0.04669097],
       [0.04568482, 0.03296549, 0.06882677, 0.02682647, 0.02996501,
        0.07791319, 0.08192724, 0.04530492, 0.10266625, 0.0589298 ,
        0.07926102, 0.09818587, 0.05211449, 0.06563818, 0.07404153,
        0.05974901],
       [0.05294081, 0.03399593, 0.06246865, 0.05362319, 0.04215889,
        0.13148265, 0.04610087, 0.06178078, 0.05657477, 0.03980442,
        0.11232825, 0.07897851, 0.03860776, 0.06544577, 0.06100322,
        0.06270555],
       [0.05987815, 0.06911524, 0.07219891, 0.06626628, 0.07097723,
        0.06333521, 0.04333451, 

In [ ]:
m.get_coherence_per_topic()

In [ ]:
len(ds.dataset.to_list())
list(filter(lambda x: x.any() is None or len(x) < 1, ds.dataset.to_list()))

# Focus on learning rate

In [ ]:
# We fix other params and now focus entirely on lr.
# We have already a "promising" range defined.
# We look in that space so we redefine lr on ABAERandomHyperparametersSelectionWrapper

## Best found model training:

## See if the Hp tuning really improved upon our results:
We used SGD anda learned its parameters under the assumption that we would do better. <br>
Let's see if it really is the case, or we just wasted time.

For comparison we use Adam that has the advantage of being robust enough without parameter scouting.

In [ ]:
#todo

## Test accuracy on small test sample we filled out

### Test set definition

In [ ]:
from core.pre_processing import PreProcessingService

# We take around 1k records that were not seen yet from the model and label them by hand.
dataset = pd.read_csv("../data/corpus.csv")
# Handle game names. Or not? I don't need the full pipeline I guess. todo
game_names = pd.read_csv("../resources/2024-08-18.csv")['Name']
game_names = pd.concat([game_names, pd.Series(["Quick", "Catan"])], ignore_index=True)
document_game_names = game_names.swifter.apply(lambda x: nlp(x)).tolist()

pipeline = PreProcessingService.full_pipeline(document_game_names, "../data/processed-dataset/full")

# Extract 1k from dataset that are not in 200k
train_ds = pd.read_csv("../output/dataset/pre-processed/200k.preprocessed.csv")

# Take top 2k. (We will select some good ones and reduce the number to 1k)
test_set = dataset[~dataset["comments"].isin(train_ds["comments"])]

We have to use labels:

In [ ]:
labels = {
    '0': "Luck/Alea",
    '1': 'Bookkeeping',
    '2': 'Downtime',
    '3': 'Interaction',
    '4': 'Bash',
    '5': 'Complicated/Complex',  # I could watch weight to see if there is a ratio relation.
    '6': 'Misc'
}